In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adam
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
data = pd.read_csv('twitter.csv', encoding = 'latin1') # çok fazla karakter olduğundan latin eklendi

In [3]:
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [4]:
print(data)

        _unit_id  _golden _unit_state  _trusted_judgments _last_judgment_at  \
0      815719226    False   finalized                   3    10/26/15 23:24   
1      815719227    False   finalized                   3    10/26/15 23:30   
2      815719228    False   finalized                   3    10/26/15 23:33   
3      815719229    False   finalized                   3    10/26/15 23:10   
4      815719230    False   finalized                   3     10/27/15 1:15   
...          ...      ...         ...                 ...               ...   
20045  815757572     True      golden                 259               NaN   
20046  815757681     True      golden                 248               NaN   
20047  815757830     True      golden                 264               NaN   
20048  815757921     True      golden                 250               NaN   
20049  815757985     True      golden                 249               NaN   

       gender  gender:confidence profile_yn  profil

In [5]:
data.shape

(20050, 26)

In [6]:
data1 = pd.concat([data.gender,data.description], axis = 1) # gender ve description kısmını alacağız sadece.Tanımlanmamış cinsiyetleri atacağız
df = data1.copy()
df.head()

,gender,description
0,male,i sing my own rhythm.
1,male,I'm the author of novels filled with family dr...
2,male,louis whining and squealing and all
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe..."
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...


In [7]:
df.dropna(inplace = True) # boş olanların verisetinden silinmesi

In [8]:
df.reset_index(drop = True, inplace = True) # Silme işleminden sonra indekslerin yeri değişti bunun düzeltilmesi
df

,gender,description
0,male,i sing my own rhythm.
1,male,I'm the author of novels filled with family dr...
2,male,louis whining and squealing and all
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe..."
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...
...,...,...
16219,female,(rp)
16220,male,"Whatever you like, it's not a problem at all. ..."
16221,male,#TeamBarcelona ..You look lost so you should f...
16222,female,Anti-statist; I homeschool my kids. Aspiring t...


In [9]:
# gender değişkeni kategorik değişken bilgisayarın anlayacağı şekile getirmemiz gerek yani 0 ve 1'e döndürmemiz gerek
# female = 1, male = 0
df.gender = [1 if i == "female" else 0 for i in df.gender]
df.head()

,gender,description
0,0,i sing my own rhythm.
1,0,I'm the author of novels filled with family dr...
2,0,louis whining and squealing and all
3,0,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe..."
4,1,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...


In [10]:
# Etiketleri ve yorumları ayırıp liste içine alınması
target = df.gender.values.tolist()
data = df.description.values.tolist()

In [11]:
target

[0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,


In [12]:
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

ps = PorterStemmer()
stop = nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\taner\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
new_data = []
for expression in data:
    expression = re.sub("[^a-zA-Z]"," ",expression) # Her bir ifadenin a-z ye ve A-Z dışındakilerinden silinmesi
    expression = expression.lower() # Her bir ifadenin küçültülmesi
    expression = expression.split() # Her bir ifadenin kelime kelime ayrılması
    expression = [ps.stem(i) for i in expression if not i in set(stopwords.words('english'))] # Her bir ifadenin stopwordslerden arındırılması
    expression = " ".join(expression) # Her bir ifadenin birleştirilip tekrardan cümle haline gelmesi
    new_data.append(expression) #Her bir cümlenin listeye eklenmesi
new_data

['sing rhythm',
 'author novel fill famili drama romanc',
 'loui whine squeal',
 'mobil guy er shazam googl kleiner perkin yahoo sprint pc airtouch air forc stanford gsb uva dad husband brother golfer',
 'ricki wilson best frontman kaiser chief best band xxxx thank kaiser chief incred year gig memori cherish alway xxxxxxx',
 'know',
 'global marketplac imag video music share photo inspir design tip video creativ commun',
 'secret get ahead get start',
 'pll fan crazi mcd ramen bae',
 'renaiss art historian univers nottingham fuell haribo partial coffe soft spot renaiss china nation teach fellow',
 'clean food tast great provid energi nutrient guilt granola vegan paleo friendli option cert organ gf kosher',
 'highli extraordinari auction',
 'senior xi xii mmxiv',
 'come join fastest blog network onlin today http co mfpa vgk http co mpuuqtyf g cover credit repair credit card bankruptci',
 'im p bo burnham disney world',
 'jmkm',
 'enthusiast f fan model collector music fan film fanat als

In [14]:
print("1.data :",data[0])
print("2.data :",new_data[0])

1.data : i sing my own rhythm.
2.data : sing rhythm


In [15]:
# Verilerin %80 eğitim %20 test verileri olacak.Önce verilerin %80 ne kadar ettiğini hesaplayacağız sonrasında oradan keseceğiz
cutoff = int(len(data) * 0.80) # == 12.979
x_train, x_test = new_data[:cutoff], new_data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [16]:
x_test

['offici twitter belfast telegraph sport news monitor',
 'ford rs owner fan thing ford rs keen golfer big fan sport motorsport',
 'becom weak sad superior victori inde true believ',
 'noth show strength meet pain compass',
 'luca wyrsch network enterpris risk manag consult actuari',
 'say photograph love polit music footbal feelthebern corbyn pm instagram http co gtbajewcju',
 'http co herwedjppt lo angel laker news',
 'move keep move',
 'cant laugh bag chip never stir paint dirti stick love music',
 'lezzz get onnnn',
 'count bless',
 'team fake hipster stalk austin blogger',
 'ig meagan tong',
 'helium way funnier polonium design univers chase dragon',
 'porno free video porno grati pussi porn free mobil porn iphon porn sex video',
 'ever tri put puzzl togeth piec miss yeah life stuck recurs loop halamadrid pakactivist',
 'helplin promot welfar gsd breed offer health diet train advic http co dgipxerfwk',
 'sad awkward',
 'digit product develop social media market activ stock investor

In [17]:
x_train[80]

'repair make model phone tablet mp player profession repair electron equip sinc'

In [18]:
# Elimizdeki yorumları tokenleştirebiliriz
# Her yorum kelimelere ayrılacak ve kelime haznesinde her kelimeye karşılık farklı bir sayı denk gelecek
# Kelime haznemizde max 10 000 kelime olsun
num_words = 10000 # En sık geçen 10 000 kelime alınacak
tokenizer = Tokenizer(num_words=num_words)

In [19]:
tokenizer.fit_on_texts(new_data)

In [20]:
# En çok bulunan kelime en başa gelecektir
print(tokenizer.word_index)
# JupyterLab 10.000 kelimenin hepsini göstermeyecek onun yerine 1000 kelime gösterecek

{'co': 1, 'http': 2, 'love': 3, 'life': 4, 'follow': 5, 'news': 6, 'music': 7, 'tweet': 8, 'like': 9, 'live': 10, 'fan': 11, 'world': 12, 'make': 13, 'one': 14, 'time': 15, 'sport': 16, 'com': 17, 'writer': 18, 'book': 19, 'busi': 20, 'twitter': 21, 'get': 22, 'lover': 23, 'game': 24, 'best': 25, 'us': 26, 'thing': 27, 'new': 28, 'work': 29, 'market': 30, 'girl': 31, 'god': 32, 'ig': 33, 'free': 34, 'day': 35, 'instagram': 36, 'account': 37, 'design': 38, 'go': 39, 'peopl': 40, 'good': 41, 'know': 42, 'artist': 43, 'student': 44, 'help': 45, 'year': 46, 'media': 47, 'updat': 48, 'want': 49, 'social': 50, 'art': 51, 'also': 52, 'friend': 53, 'alway': 54, 'travel': 55, 'offici': 56, 'find': 57, 'r': 58, 'famili': 59, 'person': 60, 'video': 61, 'author': 62, 'need': 63, 'play': 64, 'profession': 65, 'manag': 66, 'univers': 67, 'commun': 68, 'gmail': 69, 'e': 70, 'anim': 71, 'snapchat': 72, 'man': 73, 'beauti': 74, 'food': 75, 'enthusiast': 76, 'im': 77, 'footbal': 78, 'blogger': 79, 'supp

In [21]:
# Liste içerisinde tüm yorumları token olarak toplayalım
# Bizim daha önceden x_train olarak aldığımız veriler str şekilde bunları tokenlaştıracağız
x_train_tokens = tokenizer.texts_to_sequences(x_train)

In [22]:
x_train_tokens

[[776, 6379],
 [62, 1011, 2529, 59, 1136, 969],
 [595, 6380, 9651],
 [472,
  182,
  802,
  9652,
  473,
  9653,
  9654,
  1283,
  4968,
  917,
  9655,
  753,
  1336,
  3086,
  9656,
  4106,
  198,
  153,
  622,
  2321],
 [3087,
  3088,
  25,
  9657,
  6381,
  524,
  25,
  458,
  4969,
  396,
  6381,
  524,
  1643,
  46,
  3089,
  1222,
  4970,
  54,
  9658],
 [42],
 [315, 1999, 829, 61, 7, 239, 478, 138, 38, 328, 61, 267, 68],
 [417, 22, 2530, 22, 185],
 [4107, 11, 459, 6382, 6383, 777],
 [3546,
  51,
  1337,
  67,
  2777,
  6384,
  9659,
  9660,
  251,
  2531,
  2322,
  3546,
  2323,
  217,
  708,
  1183],
 [1284,
  75,
  1047,
  92,
  103,
  1137,
  9661,
  4971,
  6385,
  693,
  4108,
  830,
  3090,
  4972,
  364,
  2778,
  6386],
 [803, 2532, 2000],
 [287, 4109, 4973, 6387],
 [94,
  429,
  2533,
  104,
  199,
  111,
  311,
  2,
  1,
  9662,
  9663,
  2,
  1,
  9664,
  229,
  300,
  970,
  2144,
  970,
  637,
  6388],
 [77, 86, 3091, 9665, 658, 12],
 [9666],
 [76, 121, 11, 294, 1012

In [23]:
# Daha önceden bu durumda iken;
x_train[80]

'repair make model phone tablet mp player profession repair electron equip sinc'

In [24]:
x_train_tokens[80]

[2144, 13, 294, 1052, 9754, 2153, 235, 65, 2144, 1289, 2154, 141]

In [25]:
print(" Sentence :",x_train[80],"\n","Token :",x_train_tokens[80])

 Sentence : repair make model phone tablet mp player profession repair electron equip sinc 
 Token : [2144, 13, 294, 1052, 9754, 2153, 235, 65, 2144, 1289, 2154, 141]


In [26]:
print(" Sentence :",x_train[20],"\n","Token :",x_train_tokens[20])

 Sentence : start 
 Token : [185]


In [27]:
# Aynı şekilde test verilerinide tokenlaştırıyoruz
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [28]:
# Token işlemleri bittikten sonra yorumlara birde padding ekleyeceğiz
# RNN ile oluşturduğumuz modellere belirli boyutta inputlar veririz sinir ağlarında bu genelde böyledir. Hep belirli boyutlardaki veriler ile çalışılır
# Elimizdeki datasetindeki yorumlarda hepsi farklı sayılarda kelimelerden oluşuyor. Farklı sayılarda kelimeleri RNN'e input olarak veremeyiz
# Bu yüzden tüm yorumları aynı boyuta getirmemiz gerek
# Bunu yapmanın basit bir yolu var kendimiz belirli bir boyut belirleceğiz ve eğer bir yorum bu boyuttan düşük boyutluysa eksik kalan yerlere 0 ekleyeceğiz. Eğer belirlediğimiz boyuttan büyükse de yorumun bir kısmını silip belirlediğimiz boyuta getireceğiz

In [29]:
# Veri setindeki istatistikleri görmek için her yorumda kaç tane token bulunduğu hesaplayıp bir listeye yazalım
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens] # Liste üreteçleri
num_tokens = np.array(num_tokens) # Bu liste üzerinde işlem yapmayı kolaylaştırmak için listeyi numpy arrayine dönüştürüldü 

In [30]:
print(" Average of the Token :",np.mean(num_tokens),"\n","Max of the Token :",np.max(num_tokens),"\n","Min of the Token :",np.min(num_tokens))

 Average of the Token : 8.112241124260356 
 Max of the Token : 32 
 Min of the Token : 0


In [31]:
# Listedeki en büyük elemanın indeksi
np.argmax(num_tokens) # Datasetindeki en uzun yorum

8211

In [32]:
x_train[8211]

'kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath kath'

In [33]:
# Networke verilerin verilerin belli bir boyutta olması gerktiğini daha önceden belirtmiştik. Tüm yorumların boyutunu eşitlemek için padding kullanıyoruz
# Bu boyutu belirlerken kafamıza göre bir sayı belirleyebiliriz,veya tüm yorumların boyutunu en fazla token bulunan yoruma eşitleyebiliriz, veya ortalama kullanılabilir
# Daha da iyisi hem hafızayı yormayacak hem biraz matematik kullanarak hemde verilerin çoğunu kapsayacak bir sayı belirleyelim;
# Bu sayı sayesinde çok fazla kelimeyi yorumlarda çıkarmayacağız.Kısa cümlelere de çok fazla padding eklenemeyecek
max_tokens = int(np.mean(num_tokens) + (2 * np.std(num_tokens)))
print(max_tokens)
# Networke 18 tokenli vektörler vereceğiz

18


In [37]:
np.sum(num_tokens < max_tokens) / len(num_tokens)
# yorumların % 96'sında 18 taneden az token bulunuyor.Yorumların %95'ine 0 ekleyerek token sayısını 18e çıkaracağız. Sadece %5 lik kısımda bir veri kaybımız olacak

0.9659763313609467

In [50]:
# Önce eğitim setindeki tokenlere padding ekleyelim
x_train_pad = pad_sequences(x_train_tokens, maxlen = max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen = max_tokens)
print(" X Train Shape :",x_train_pad.shape,"\n","X Test Shape :",x_test_pad.shape) # Artık her yorum 18 uzunluğunda

 X Train Shape : (12979, 18) 
 X Test Shape : (3245, 18)


In [39]:
x_train_pad[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,  776, 6379])

In [40]:
x_train_pad.shape

(12979, 18)

In [41]:
x_test_pad.shape

(3245, 18)

In [65]:
print("Without Padding :", np.array(x_train_tokens[80]),"///", "Lenght :" ,len(np.array(x_train_tokens[80])), 
      "\n", "-" * 90, "\n"
      "Padding :", x_train_pad[80], "///", "Lenght :" ,len(x_train_pad[80])
)

Without Padding : [2144   13  294 1052 9754 2153  235   65 2144 1289 2154  141] /// Lenght : 12 
 ------------------------------------------------------------------------------------------ 
Padding : [   0    0    0    0    0    0 2144   13  294 1052 9754 2153  235   65
 2144 1289 2154  141] /// Lenght : 18


In [52]:
# Aynı yorumun padding uygulanmış haline bakalım; 0 eklenerek boyutu 59'a tamamlanmış
x_train_pad[80]

array([   0,    0,    0,    0,    0,    0, 2144,   13,  294, 1052, 9754,
       2153,  235,   65, 2144, 1289, 2154,  141])

In [66]:
# Keras içinde stringi verip tokenleri görebiliyoruz ancak tokenleri verip stringi göremiyoruz bunun için bir fonksiyon yazacağız
idx = tokenizer.word_index # word_index içinde sözlük olarak kelimeler ve kelimelerin sayısal olarak tokenleri bulunuyor. Biz bu sözlüğü tersine çevireceğiz 
idx

{'co': 1,
 'http': 2,
 'love': 3,
 'life': 4,
 'follow': 5,
 'news': 6,
 'music': 7,
 'tweet': 8,
 'like': 9,
 'live': 10,
 'fan': 11,
 'world': 12,
 'make': 13,
 'one': 14,
 'time': 15,
 'sport': 16,
 'com': 17,
 'writer': 18,
 'book': 19,
 'busi': 20,
 'twitter': 21,
 'get': 22,
 'lover': 23,
 'game': 24,
 'best': 25,
 'us': 26,
 'thing': 27,
 'new': 28,
 'work': 29,
 'market': 30,
 'girl': 31,
 'god': 32,
 'ig': 33,
 'free': 34,
 'day': 35,
 'instagram': 36,
 'account': 37,
 'design': 38,
 'go': 39,
 'peopl': 40,
 'good': 41,
 'know': 42,
 'artist': 43,
 'student': 44,
 'help': 45,
 'year': 46,
 'media': 47,
 'updat': 48,
 'want': 49,
 'social': 50,
 'art': 51,
 'also': 52,
 'friend': 53,
 'alway': 54,
 'travel': 55,
 'offici': 56,
 'find': 57,
 'r': 58,
 'famili': 59,
 'person': 60,
 'video': 61,
 'author': 62,
 'need': 63,
 'play': 64,
 'profession': 65,
 'manag': 66,
 'univers': 67,
 'commun': 68,
 'gmail': 69,
 'e': 70,
 'anim': 71,
 'snapchat': 72,
 'man': 73,
 'beauti': 74,
 '

In [67]:
inverse_map = dict(zip(idx.values(), idx.keys())) # Sayılar anahtarımız kelimeler ise değerler olacak
inverse_map

{1: 'co',
 2: 'http',
 3: 'love',
 4: 'life',
 5: 'follow',
 6: 'news',
 7: 'music',
 8: 'tweet',
 9: 'like',
 10: 'live',
 11: 'fan',
 12: 'world',
 13: 'make',
 14: 'one',
 15: 'time',
 16: 'sport',
 17: 'com',
 18: 'writer',
 19: 'book',
 20: 'busi',
 21: 'twitter',
 22: 'get',
 23: 'lover',
 24: 'game',
 25: 'best',
 26: 'us',
 27: 'thing',
 28: 'new',
 29: 'work',
 30: 'market',
 31: 'girl',
 32: 'god',
 33: 'ig',
 34: 'free',
 35: 'day',
 36: 'instagram',
 37: 'account',
 38: 'design',
 39: 'go',
 40: 'peopl',
 41: 'good',
 42: 'know',
 43: 'artist',
 44: 'student',
 45: 'help',
 46: 'year',
 47: 'media',
 48: 'updat',
 49: 'want',
 50: 'social',
 51: 'art',
 52: 'also',
 53: 'friend',
 54: 'alway',
 55: 'travel',
 56: 'offici',
 57: 'find',
 58: 'r',
 59: 'famili',
 60: 'person',
 61: 'video',
 62: 'author',
 63: 'need',
 64: 'play',
 65: 'profession',
 66: 'manag',
 67: 'univers',
 68: 'commun',
 69: 'gmail',
 70: 'e',
 71: 'anim',
 72: 'snapchat',
 73: 'man',
 74: 'beauti',
 7

In [98]:
# Bunu text üzerinde yapmak için bir fonksiyon oluşturalım
def text_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token != 0]
    text = ' '.join(words)
    return text

In [99]:
print(" Henüz tokenleştirilmeyen 80. tweet ==>",x_train[80],"\n\n",
      "Tokenleştirilmiş 80. tweet ==>",x_train_tokens[80],"\n\n",
      "Fonksiyon ile tokenleri  texte çevirilmiş 80. tweet ==>", text_to_string(x_train_tokens[80]))

 Henüz tokenleştirilmeyen 80. tweet ==> repair make model phone tablet mp player profession repair electron equip sinc 

 Tokenleştirilmiş 80. tweet ==> [2144, 13, 294, 1052, 9754, 2153, 235, 65, 2144, 1289, 2154, 141] 

 Fonksiyon ile tokenleri  texte çevirilmiş 80. tweet ==> repair make model phone tablet mp player profession repair electron equip sinc


In [100]:
# Henüz tokenleştirilmeyen 800. tweet alalım;
x_train[80]

'repair make model phone tablet mp player profession repair electron equip sinc'

In [101]:
# Tokenleştirilmiş 80. tweet
x_train_tokens[80]

[2144, 13, 294, 1052, 9754, 2153, 235, 65, 2144, 1289, 2154, 141]

In [102]:
# Fonksiyonumuz kullanarak yazdığımız bu cümlenin tokenlerini texte çevirelim. Aynı yazı elde edilirse fonksiyon başarılıdır
text_to_string(x_train_tokens[80])
# Fakat bazı kelimeler çıkarılmış. Bunun sebebi tokenleştirme yaparken sık kullanılan 10000 kelimeyi almıştık eğer bir kelime 10 000 kelime arasında yoksa çıkarılıyor. Örneğin 'bilgisyarım ' kelimesi yok

'repair make model phone tablet mp player profession repair electron equip sinc'

In [103]:
# Önce embedding matrisi oluşturup 3 katlı Gru ile eğitim yapacağız
# Gru sonunda tek bir nöron olacak ve bu nöronun ürettiği değer 1'e yakınsa erkek olumlu, 0 ise kadın

In [104]:
# Ardaşık bir model oluşturuyoruz
model = Sequential()

In [105]:
# Modelimizde olmasını istediğimiz her şeyi bu modele katman katman ekleyeceğiz 
# Embedding matrisini oluşturmadan önce büyüklüğünü belirleyelim
embedding_size = 50 # Her kelimeye karşılık gelen 20 uzunluğunda bir vektör oluşturuldu

In [106]:
# Boyutlar belirlendiğine göre artık kerasta embedding layer oluşturabiliriz
# Kelime vekrtörlerini word2vec ve başka yöntemlerle oluşturuyorduk fakat bu sefer biz embedding matrisini oluştururken word2vec vs herhangi bir şey kullanamayacağız. Kelime vektörlerini rasgele oluşturacağız
# Vektörler rasgele oluşturulsa bile gayet iyi sonuçlar çıkabiliyor
# Modeli eğitirken kelime vektörlerini de eğiteceğiz
# word2vec ile yapılacak olanı daha sonradan ekeleyeceğiz  

In [107]:
np.argmin(num_tokens)

67

In [108]:
x_train[17]

'artisan special paper mach print make fibr art art teacher cat devote find page fb http co g bstg icv'

In [109]:
# Embedding Layerı modele .add ile ekliyoruz
# input_dim = kelime sayıları(10.000)
# output_dim = kelime vektörlerinin uzunluğu 
# Sonuç olarak (10.000,50) matris oluşturuyoruz ve rasgele sayılardan oluşacak
# input_length = max_tokens ==> 18
model.add(Embedding(input_dim = num_words,
                    output_dim = embedding_size,
                    input_length = max_tokens,
                    name = 'embedding_layer'
                   )) 

In [110]:
# Örneğin 'bu ürün süper' olarak bir yorum gelsin. 10.000 kelime içerisinden bu cümle içerisindeki kelimelere karşılık gelen vektörleri alacağız
# Yani 'bu','ürün','süper' vektörlerini alıp bir sonraki layera input olarak vereceğiz. 10.000 kelimenin hepsini birden output olarak vermiyor
# Bu layer sadece gelen inputların vektörlerini output olarak veriyor
# Embedding layerin outuputu bir sonraki layerin inputu olacak

In [111]:
# units = 16 gru layersın nöron sayısı. Bu layerda 16 adet outuput verilecek
# output olarak sqeunecesin tamamı döndürülüyor. False olsaydı sadece son output döndürelecekti. Bir sonraki layera gru ekleyeceğimiz için True dememiz gerek. Eğer bir sonraki layer 1 nörondan oluşsaydı bu false olurdu
model.add(GRU(units = 16, return_sequences = True))
model.add(GRU(units = 8, return_sequences = True))
model.add(GRU(units = 4)) # return_sequences = False olması gerek 4 nöron olacaksa default olarak false geliyor.
model.add(Dense(1, activation = 'sigmoid')) # Tek bir nörondan oluştuğu için = 1

In [112]:
# ---------------------------------ÖZET----------------------------------------------------------
# Sinir ağımıza başlangıçta input olarak bir yorum veriyoruz ve bu yorum tokenlerden oluşuyor
# Yorumun içinde her kelimeye karşılık gelen bir kelime vektörü var
# Embedding Layerda yorumun içerisindeki kelimelere karşılık gelen vektörler bulunuyor ve bu vektörler bir sonraki layera yani GRU Layerina output olarak veriliyor
# Alınan kelime vektörleri 3 tane GRU Layerında işlemden geçiriliyor ve en son output layerindaki tek bir nörona input olarak değerler gönderiliyor
# Output layerindaki nöronumuzda değerleri tek bir değere dönüştürüp sigmoid aktivasyon fonksiyonundan geçirerek bize 0 ile 1 arasında değer döndürüyor. Bu değer modelimizin o yorum hakkındaki tahmini oluyor.
# Eğer çıkan sonuç 1'e yakın olursa modelimiz aldığı yorumun olumlu olduğunu düşünüyor. Eğer 0 isede olumsuz

In [114]:
from tensorflow.python.keras.optimizer_v2.adam import Adam  # Model.compile çalılmadı yen version

In [115]:
# Optimizasyon algoritması için Adam
optimizer = Adam(lr = 1e-3) # lr = learning rate , 1e-3 == 0,001

In [116]:
# Model derlenmesi
model.compile(loss = 'binary_crossentropy', # sadece iki sınıf olduğu için binary_crossentropy daha fazla sınfılar içinde catogorical_crossentropy kullanılır
              optimizer = optimizer,
              metrics = ['accuracy']) # Modelimizin başarı oranını görmek için

In [117]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 18, 50)            500000    
_________________________________________________________________
gru (GRU)                    (None, 18, 16)            3216      
_________________________________________________________________
gru_1 (GRU)                  (None, 18, 8)             600       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 503,977
Trainable params: 503,977
Non-trainable params: 0
_________________________________________________________________


In [118]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [119]:
model.fit(np.array(x_train_pad), np.array(y_train), epochs = 5, batch_size = 256)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
51/51 [==============================] - 6s 29ms/step - loss: 0.6664 - accuracy: 0.6455
Epoch 2/5
51/51 [==============================] - 1s 28ms/step - loss: 0.5663 - accuracy: 0.6961
Epoch 3/5
51/51 [==============================] - 1s 28ms/step - loss: 0.4321 - accuracy: 0.8027
Epoch 4/5
51/51 [==============================] - 1s 29ms/step - loss: 0.3595 - acc

In [120]:
result = model.evaluate(np.array(x_test_pad), np.array(y_test))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
102/102 [==============================] - 1s 4ms/step - loss: 0.7218 - accuracy: 0.6595


In [121]:
print("Loss Değeri :",result[0])
print("İsabet Oranı :",result[1])
# % 95 oranında isabet 

Loss Değeri : 0.7218081951141357
İsabet Oranı : 0.659476101398468


In [122]:
# İlk 1000 yorumun tahminine bakalım;
y_pred = model.predict(x = np.array(x_test_pad[0:1000]))
y_pred = y_pred# Böyle çalışıyor   ----------???belki sonuna .T[0]------------????

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [123]:
y_pred

array([[0.05653781],
       [0.05641809],
       [0.49981263],
       [0.2108702 ],
       [0.06223899],
       [0.5743907 ],
       [0.11559156],
       [0.06977463],
       [0.15298495],
       [0.6474426 ],
       [0.7547081 ],
       [0.06731084],
       [0.8252419 ],
       [0.52048033],
       [0.05645061],
       [0.05740777],
       [0.05693027],
       [0.86541426],
       [0.05683374],
       [0.3550763 ],
       [0.20240164],
       [0.8105227 ],
       [0.05703411],
       [0.58657926],
       [0.90672827],
       [0.05848229],
       [0.57866126],
       [0.7187561 ],
       [0.13367376],
       [0.05811208],
       [0.8012363 ],
       [0.456131  ],
       [0.7338717 ],
       [0.9188781 ],
       [0.37579083],
       [0.8675786 ],
       [0.72925425],
       [0.86608934],
       [0.6435361 ],
       [0.0565182 ],
       [0.8828205 ],
       [0.23464337],
       [0.07190496],
       [0.8218813 ],
       [0.53841966],
       [0.05706954],
       [0.77034974],
       [0.391

In [124]:
cls_pred = np.array([1.0 if p > 0.5 else 0.0 for p in y_pred])

In [125]:
# Modelin tahminlerini gerçek değerler ile karşılaştırmak için test setinin labellarının ilk 1000 elemanını numpy array olarak alalım
cls_true = np.array(y_test[0:1000])

In [126]:
# Modelin tahmini ve gerçekte olması gereken sınıfları karşılaştırabiliriz. Numpy ile iki vektörü karşılaştırmak oldukça kolay
# Elimizdeki iki vektörü karşılaştıracağız ve modelimizin yanlış tahminlerini vektör içerisinde saklayacağız
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [127]:
len(incorrect)
# 1000 yorum içinden 326 tanesi yanlış bilinmiş

340

In [128]:
# Yanlış bilinen ilk yorumun indeksi
idx = incorrect[0]
idx

5

In [129]:
# 2.yorumu modelimiz yanlış tahmin etmiş
text = x_test[idx]
text

'say photograph love polit music footbal feelthebern corbyn pm instagram http co gtbajewcju'

In [130]:
# Bu yorum için tahmini
# Eğitimde 0.51 çıkmıştı
y_pred[idx]

array([0.5743907], dtype=float32)

In [131]:
cls_true[idx] # Olumsuz

0

In [132]:
text1 = "Last chapter! Enjoy" # JK Rowling
text2 = "Ah, I remember Leah's picture! So beautiful. If I'd been judging the competition there would have been 1000x as many pictures..." # JK Rowling
text3 = "Someone suggested changing Dogecoin fees based on phases of the moon, which is pretty awesome haha" # Elon Musk
text4 = "To be clear, I strongly believe in crypto, but it can’t drive a massive increase in fossil fuel use, especially coal" # Elon Musk
texts = [text1,text2,text3,text4]

In [133]:
tokens = tokenizer.texts_to_sequences(texts)

In [134]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape

(4, 18)

In [135]:
for i in range(1,5):
    print("text :", i, "basarı sonucu =", model.predict(tokens_pad)[i-1][0])

text : 1 basarı sonucu = 0.82969654
text : 2 basarı sonucu = 0.85336864
text : 3 basarı sonucu = 0.5210408
text : 4 basarı sonucu = 0.14306393
